## 1. Introduction & Research Questions

Diabetes is a major public health concern associated with increased morbidity, mortality, and healthcare costs. Understanding demographic and behavioral factors associated with the widespread of diabetes can help inform prevention strategies and public health interventions.

This project explores relationships between diabetes diagnosis and selected risk factors using public health survey data. The analysis focuses on exploratory data analysis and statistical inference rather than predictive modeling.

### Research Questions
The primary questions addressed in this analysis are:

1. Is body mass index (BMI) significantly associated with diabetes prevalence?
2. Do physically active individuals exhibit lower rates of diabetes compared to inactive individuals?
3. Does diabetes prevalence differ significantly across age groups?
4. Is income level associated with diabetes diagnosis?

## 2. Data Overview

The dataset used in this project is derived from a public health behavioral survey and includes individual-level responses related to diabetes diagnosis, demographic characteristics, and health-related behaviors.
"Diabetes Health Indicators Dataset" - https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset

### Key Variables
- **Diabetes diagnosis** (binary outcome)
- **Body Mass Index (BMI)**
- **Age group**
- **Physical activity status**
- **Smoking status**
- **Income level**
- **Education level**

The data consists of self-reported survey responses and is observational in nature.

## 3. Data Cleaning & Preparation

## 4. Exploratory Data Analysis

## 5. Statistical Analysis

## 6. Key Insights

## 7. Limitations & Next Steps